## Packages

In [ ]:
# Packages Used

#------------------------To solve the ODE----------------------------
using DifferentialEquations, LSODA

#------------------------Writing Data to CSV file--------------------
using CSV, Tables

# #-----------------------For Benchmarking the ODE solver------------------
# using BenchmarkTools
# using Logging: global_logger
# using TerminalLoggers: TerminalLogger
# global_logger(TerminalLogger())   

## Defining the System

In [ ]:

#---------------------------System Parameters-----------------------------------

m1 = 4.8e-11
m2 = 4.8e-14
D  = 1e5 
a0 = 2.88e-8                 # 100*r_isco
e0 = 0.

#---------------------------Mass Array------------------------------------------

m_total = m1+m2
mu = (m1*m2)/(m1+m2)
mass=[m1,m2,m_total,mu]

#---------------------------Initial Conditions----------------------------------

function initial_conditions(mass, a0, e0, phi0 = 0.)
    """
    Calculate the initial conditions for a Keplerian orbit with parameters a, e
    """
    r0 = a0 * (1. - e0^2) / (1. + e0 * cos(phi0))
    dphi0 = sqrt(mass[3] * a0 * (1. - e0^2)) / r0^2
    dr0 = a0* (1. - e0^2) / (1 + e0*cos(phi0))^2 * e0 *sin(phi0)*dphi0
    return [r0, phi0, dr0, dphi0]
end

u0 = initial_conditions(mass, a0, e0)

#-------------------------Halo Models--------------------------------------------

function Spike_Halo(r, rho_spike, r_spike, alpha, r_min=0.)   # Spike Model
    
    """
    Defining the Halo Model from Eda paper [https://arxiv.org/abs/1408.3534]
    
    The density is given by:
        rho (r) = rho_spike*(r_spike/r)^(alpha)
    
    Parameters:
        rho_spike : float
            The density parameter of the spike profile
        r_spike : float
            The scale radius of the spike profile
        alpha : float
            The power-law index of the spike profile, with condition 0 < alpha < 3
    """
    
    return  ifelse.(r .> r_min, rho_spike.*(r_spike./r).^alpha, 0.)
end


function Const_Halo(r, rho_0, r_min=0.)    # Constant Density Model
    
    """
    Constant Halo with density rho_0
    """
    
    return  ifelse.(r .> r_min, rho_0, 0.)
end


#--------------------Halo Parameters from Eda paper----------------------------------

rho_spike = 1.0848e-11               # [226.*ms.solar_mass_to_pc]
alpha_spike = 7/2.8
r_spike = 0.54;


## Evolving the System

In [ ]:


#-------------------------Evolution Parameters--------------------------------------------

F0 = sqrt(mass[3]/a0^3)/2/pi
nOrbits = 1000                     # Number of Orbits
tp_per_orb = 100                   # Time-points per orbit
dp = nOrbits*tp_per_orb            # total number of data points the code runs for
t_end = nOrbits/F0


#-------------------------Evolution Function-----------------------------------------------

function Evolve(mass, u0, t_end, t_start=0., gwEmission=true, dynamicalFriction=true, PostNewtonian=true, coulombLog=3., nSteps=1000)
    
    """
    Evolve the system of differential equations from t_start to t_end with initial conditions u0
    """
    t_eval = LinRange(t_start, t_end, nSteps)
    eta = mass[4]/mass[3]
    m = mass
    tspan = (t_start,t_end)
 
    
    #---------------------------Derivative Function---------------------------------------
    
    function orbit!(du, u, m, t)
        
        # r=u[1],  phi=u[2],  dr=u[3],  dphi=u[4]
        # m = [m1,m2,m_total,mu]
        
        v = sqrt(u[3]^2 + (u[1]^2)*(u[4])^2) 
        
        P_df     = dynamicalFriction ? 4*pi*(m[2]^2 / m[4]) * (Spike_Halo(u[1],rho_spike, r_spike, alpha_spike)) * (coulombLog/v^3) : 0.
        
        P_pn_r   = PostNewtonian ? (m[3]/u[1]^2)*( (4 + 2*eta)*(m[3]/u[1]) - (3*eta +1)*(u[1]^2)*(u[4]^2) + (3 - 7*eta/2)*u[3]^2) : 0.
        
        P_pn_phi = PostNewtonian ? (m[3]/u[1]^2)*(4 - 2*eta)*u[3]*u[4] : 0.
        
        P_gw_r   = gwEmission ? ( (8/5)*m[4]*(m[3]/u[1]^3)*(2*v^2 + (8/3)*(m[3]/u[1]) )) : 0.
        
        P_gw_phi = gwEmission ? ( (8/5)*m[4]*(m[3]/u[1]^3)*(v^2 + 3*(m[3]/u[1]) )) : 0.
        
        
        du[1] = u[3]  # dr
        du[2] = u[4]  # dphi
        du[3] = -(-P_gw_r + P_df)*(u[3]) + P_pn_r - (m[3]/u[1]^2) + u[1]*(u[4]^2)  # ddr
        du[4] = -(P_gw_phi + P_df)*(u[4]) + P_pn_phi - (2*u[3]*u[4]/u[1])          # ddphi    
    end
    
    
    #---------------------Call-back functions---------------------------------------------
    
    function terminate_condition(u,t,integrator)       # condition at which the integration terminates
        u[1]< 2.0e-9
    end
    
    function terminate_affect!(integrator)
        terminate!(integrator)
    end
    
    terminate_cb =DiscreteCallback(terminate_condition,terminate_affect!)
 
    
    #-------------------------Calling the solver--------------------------------------------
    
    # call-backs do not work with lsoda() but they do work with other algorithms
    
    prob = ODEProblem(orbit!, u0, tspan, m) #, callback=terminate_cb)  # exclude the callback part when using lsoda()
    alg =  lsoda() #Vern9(lazy=false) #Feagin12() #Vern7() #lsoda()
    @time sol = solve(prob, alg, abstol=1e-14, reltol=1e-12, saveat=t_eval, dense=false, maxiters=Int(1e9)) 
    return sol
end


In [ ]:
solution = Evolve(mass,u0,t_end,0.,true,true,true,3., dp)


## Writing the Orbit data to a CSV file


In [ ]:

#----------------------------------------------------------------------------
sol_transpose = solution'
t = solution.t

A = hcat(sol_transpose[:,:],t[:])     # Stacking the matrices together

#-----------------------------------------------------------------------------
CSV.write("PN_GW_DF_Orb1k_dp100_alpha_2point50.csv", Tables.table(A), writeheader=false)